## For 'import ipynb files' - *Todo (convert .ipynb to .py)*
1. install import_ipynb, PyDrive
2. import ipynb files

In [22]:
!pip install -U -q PyDrive
!pip install import_ipynb
import import_ipynb
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# id for import files
'''
layers.ipynb : 1TE_v-Os9bcDepAWODmnDi5WuAz2tyfYn
optim.ipynb : 1kCLfroPeioTdHqogFoOkTVyFYsYe9RAh
'''
# test import
module = drive.CreateFile({'id':'1b1teolDFHMLhucgVFL_-2wjOyFoFzGmJ'})
module.GetContentFile('test_function.ipynb')
from test_function import print_test
print_test()

# import require module
module = drive.CreateFile({'id':'1TE_v-Os9bcDepAWODmnDi5WuAz2tyfYn'})
module.GetContentFile('layers.ipynb')
module = drive.CreateFile({'id':'1kCLfroPeioTdHqogFoOkTVyFYsYe9RAh'})
module.GetContentFile('optim.ipynb')

test


### GPU config
- **True** : use cupy
- **False** : use numpy

In [23]:
GPU = True

if GPU:
    import cupy as np
    np.cuda.set_allocator(np.cuda.MemoryPool().malloc)
else:
    import numpy as np

### import requirements
- layer, optim
- upgrade version of scikit-learn for use ***TimeSeriesSplit()*** -gap parameter

In [24]:
from layers import LSTM, TimeLSTM, FullyConnected
from optim import Adam

# for time series split
!pip install scikit-learn==0.24.2

### Define Custom Layer for data

In [25]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def MSE(y, t):
    return 0.5*np.mean((y-t)**2)

class ReLU:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x<=0)
        y = x.copy()
        y[self.mask] = 0

        return y
    
    def backward(self, dy):
        dy[self.mask] = 0
        dx = dy

        return dx


class TimeFC:
    def __init__(self, W, b):
        self.params = [W, b]
        self.grads = [np.zeros_like(W), np.zeros_like(b)]
        self.x = None

    def forward(self, x):
        W, b = self.params
        N, D = x.shape

        reshaped_x = x.reshape(N, -1)
        y = np.dot(reshaped_x, W) + b
        
        self.x = x
        y = y.reshape(N, -1)
        return y

    def backward(self, dy):
        W, b = self.params
        x = self.x
        N, D = x.shape

        dy = dy.reshape(N, -1)
        reshaped_x = x.reshape(N, -1)

        db = np.sum(dy, axis=0)
        dx = np.matmul(dy, W.T)
        dW = np.matmul(reshaped_x.T, dy)
        
        dx = dx.reshape(*x.shape)

        self.grads[0][...] = dW
        self.grads[1][...] = db

        return dx

class TimeMSE:
    def __init__(self):
        self.params, self.grads = [], []
        self.activation = ReLU()
        self.cache = None

    def forward(self, xs, ts):
        N, V = xs.shape
        xs = xs.reshape(N, V)
        xs = self.activation.forward(xs)
        ts = ts.reshape(N, V)

        loss = MSE(xs, ts)
        self.cache = (ts, xs, (N, V))

        return loss

    def backward(self, dy = 1):

        ts, xs, (N,  V) = self.cache
        
        dx = dy * (xs - ts) / (N)

        dx = self.activation.backward(dx)
        dx = dx.reshape(N , V)
        return dx

### Model Architecture for Time Series Data
- use **LSTM** Layer
- LSTM(100) - FC(100) - FC(64)
- Add ***fit()*** Method for validation

In [26]:
class LstmModelReg:
    def __init__(self, time_size, hidden_size, feature_size):
        T, H, F = time_size, hidden_size, feature_size
        H2 = 64
        rand = np.random.randn

        # weights (Xavier)
        lstm_Wx = (rand(F, 4*H)/ np.sqrt(F)).astype('f')
        lstm_Wh = (rand(H, 4*H)/ np.sqrt(H)).astype('f')
        lstm_b = np.zeros(4*H).astype('f')

        # He initialize
        fc_W1 = (rand(H, H2)/ np.sqrt(H/2)).astype('f')
        fc_b1 = np.zeros(H2).astype('f')

        fc_W2 = (rand(H2, 1)/ np.sqrt(H2/2)).astype('f')
        fc_b2 = np.zeros(1).astype('f')

        # layer
        self.layers = [
            TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=True),
            TimeFC(fc_W1, fc_b1),
            TimeFC(fc_W2, fc_b2)
        ]
        self.loss_layer = TimeMSE()

        self.params, self.grads = [], []

        for layer in self.layers:
            self.params += layer.params
            self.grads += layer.grads


    def predict(self, xs):
        xs = np.array(xs)
        for layer in self.layers:
            xs = layer.forward(xs)
        return xs

    def forward(self, xs, ts):
        xs = np.array(xs)
        ts = np.array(ts)
        for layer in self.layers:
            xs = layer.forward(xs)
        loss = self.loss_layer.forward(xs, ts)
        return loss

    def backward(self, dy = 1):
        dy = self.loss_layer.backward(dy)
        for layer in reversed(self.layers):
            dy = layer.backward(dy)
        return dy

    def fit(self, train_X=None, train_y=None,learning_rate=0.01, epochs=10, batch_size=32, verbose=0):
        optimizer = Adam(learning_rate)

        data_size = train_X.shape[0]
        max_iters = data_size//batch_size

        for epoch in range(1, epochs+1):
            idx = numpy.random.permutation(numpy.arange(data_size))
            train_X = train_X[idx]
            train_y = train_y[idx]

            epoch_loss = 0
            start_time=time.time()
            
            for iter in range(max_iters):
                batch_x = train_X[iter*batch_size:(iter+1)*batch_size]
                batch_y = train_y[iter*batch_size:(iter+1)*batch_size]

                loss = self.forward(batch_x, batch_y)
                self.backward()
                params, grads = self.params, self.grads
                optimizer.update(params, grads)

                epoch_loss += loss
            avg_loss = epoch_loss/max_iters

            if verbose:
                duration = start_time-time.time()
                print(f'epoch:{epoch}/{epochs}, 시간:{duration:.2f}[s], loss:{avg_loss:.5f}')


    def reset_state(self):
        self.layers[0].reset_state()

### Functions for Data Preprocessing and Cross Validation

In [27]:
# series data to sequence function
def series_to_sequence(dataset, time_step=1):
    num = dataset.shape[1]      # features num
    df = pd.DataFrame(dataset)
    cols, names = list(), list()
    # sequence t-n to t-1
    for i in range(time_step, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(num)]

    for i in range(0, 1):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(num)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(num)]

    agg = pd.concat(cols, axis=1)
    agg.columns = names
    agg.dropna(inplace=True)
    return agg

# configuration setting
def model_config():
    # parameter for LSTM Model
    epochs = [10]
    batch_size = [64]
    learning_rate = [0.1, 0.01, 0.001]
    
    # create config data
    configs = []
    for i in epochs:
        for j in batch_size:
            for k in learning_rate:
                config = [i, j, k]
                configs.append(config)
    return configs

# fucntion for fit cnn model using configs
def model_fit(train_X, train_y, config):
    # unpack config
    n_epochs, n_batch, learning_rate = config
    model = LstmModelReg(time_size=24, hidden_size=100, feature_size=8)
    # fit model and return
    model.fit(train_X=train_X, train_y=train_y, epochs=n_epochs, 
              batch_size=n_batch, learning_rate=learning_rate)
    return model

def MAE_metric(x, t):
    return np.mean(np.abs(x-t))

def MSE_metric(x, t):
    return np.mean((x-t)**2)

### Nested Cross Validation for Time Series Data
- Data : Beijing Air Pollution

In [31]:
# dataset
import pandas as pd
import numpy
import time
from datetime import datetime

df_parser = lambda x: datetime.strptime(x, '%Y %m %d %H')    # string to datetime
# data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00381/PRSA_data_2010.1.1-2014.12.31.csv'
data_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/pollution.csv'
df = pd.read_csv(data_url, sep=',', parse_dates=[['year', 'month', 'day', 'hour']], date_parser=df_parser, index_col=0)

del df['No']
df.columns = ['pm2.5', 'dewp', 'temp', 'pres', 'cbwd','wind_speed', 'snow', 'rain']
df = df[24:]            # NaN values in first 24hours

# sklearn library for time series split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import mean_squared_error

dataset = df.values
label_encoder = LabelEncoder()
dataset[:, 4] = label_encoder.fit_transform(dataset[:, 4])  # for wind direction
dataset = dataset.astype('float')

n_inputs = 24       # input time dim
n_features = 8      # input feature dim
del_idx = n_inputs * n_features + 1
del_cols = [i for i in range(del_idx, del_idx+n_features-1)]
new_df = series_to_img(dataset, n_inputs)       # time series to img data
new_df.drop(new_df.columns[del_cols], axis=1, inplace=True)
new_df.reset_index(drop=True, inplace=True)

n_splits = 3
train_test_split = TimeSeriesSplit(n_splits=n_splits+1, gap=n_inputs).split(new_df)
next(train_test_split)

configs = model_config()
history = []
i = 1

print('config : epochs, batch_size, learning_rate')

# nested cross validation for time series model
for train_cv_indices, test_cv_indices in train_test_split:
    print(f'fold : {i}/{n_splits}')
    i+=1

    # split x, y data
    train_cv_X, train_cv_y = new_df.iloc[train_cv_indices, :-1].values, new_df.iloc[train_cv_indices,-1].values
    test_cv_X, test_cv_y = new_df.iloc[test_cv_indices, :-1].values, new_df.iloc[test_cv_indices, -1].values

    # length for validation set
    test_length = len(test_cv_X)

    # scaling data
    scaler_x = MinMaxScaler()
    train_cv_X = scaler_x.fit_transform(train_cv_X)
    test_cv_X = scaler_x.transform(test_cv_X)

    train_X, val_X = train_cv_X[:-test_length, :], train_cv_X[-test_length:, :]
    train_y, val_y = train_cv_y[:-test_length], train_cv_y[-test_length:]

    # reshape
    # inner loop
    train_X = train_X.reshape(-1, n_inputs, n_features)
    val_X = val_X.reshape(-1, n_inputs, n_features)
    train_y = train_y.reshape(-1, 1)
    val_y = val_y.reshape(-1, 1)

    # outter loop
    train_cv_X = train_cv_X.reshape(-1, n_inputs, n_features)
    test_cv_X = test_cv_X.reshape(-1, n_inputs, n_features)
    train_cv_y = train_cv_y.reshape(-1, 1)
    test_cv_y = test_cv_y.reshape(-1, 1)

    # model fit
    errors = []
    for idx, cfg in enumerate(configs):
        print(f' == train {cfg} model == ', end=' ')
        model = model_fit(train_X, train_y, cfg)
        model.reset_state()
        predicted = model.predict(val_X)
        if GPU:
            predicted = np.asnumpy(predicted)
        error = np.sqrt(MSE_metric(predicted, val_y))   # rmse
        print(f' error(RMSE):{error}')
        if errors:
            if error < min(errors):
                param = idx
        else:
            param = idx
        errors.append(error)

    history.append(errors)

    selected_model = model_fit(train_cv_X,train_cv_y, configs[param])
    selected_model.reset_state()
    predicted = selected_model.predict(test_cv_X)
    if GPU:
        predicted = np.asnumpy(predicted)
    error = np.sqrt(MSE_metric(predicted, test_cv_y))

    # model eval
    print(f'best_model => error(rmse) : {error}, param:{param}')

config : epochs, batch_size, learning_rate
fold : 1/3
 == train [10, 64, 0.1] model ==   error(RMSE):99.0890638027906
 == train [10, 64, 0.01] model ==   error(RMSE):32.98655118835107
 == train [10, 64, 0.001] model ==   error(RMSE):35.29681060201724
best_model => error(rmse) : 23.323251892834353, param:1
fold : 2/3
 == train [10, 64, 0.1] model ==   error(RMSE):127.81696520407286
 == train [10, 64, 0.01] model ==   error(RMSE):38.541398905362605
 == train [10, 64, 0.001] model ==   error(RMSE):22.743030160214285
best_model => error(rmse) : 22.56849850186303, param:2
fold : 3/3
 == train [10, 64, 0.1] model ==   error(RMSE):87.76604012206721
 == train [10, 64, 0.01] model ==   error(RMSE):23.088664983422596
 == train [10, 64, 0.001] model ==   error(RMSE):22.514452675433873
best_model => error(rmse) : 21.473849720065555, param:2


In [2]:
model_evaluation = 23.32+22.56+21.47
model_evaluation /= 3
print(f'evaluation [Mean RMSE] : {model_evaluation}')

evaluation [Mean RMSE] : 22.45


In [36]:
selected_model.reset_state()
predicted = selected_model.predict(test_cv_X)
if GPU:
    predicted = np.asnumpy(predicted)
print(f'MSE : {MSE_metric(predicted, test_cv_y)}')
print(f'RMSE : {np.sqrt(MSE_metric(predicted, test_cv_y))}')

def MAE_metric(x, t):
    return np.mean(numpy.abs(x-t))
print(f'MAE : {MAE_metric(predicted, test_cv_y)}')

MSE : 461.1262217999595
RMSE : 21.473849720065555
MAE : 13.374773331339261
